<a href="https://colab.research.google.com/github/joshuwaifo/A-Bible-Pre-trained-Transformer-Model/blob/main/NASB_Bible_Tokenised_BibleGPT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preliminaries:

Download the Bible text file nasb (dataset)

Open the nasb.txt file

Read in all the text as a string




In [ ]:
!wget https://raw.githubusercontent.com/tushortz/variety-bible-text/master/bibles/nasb.txt

--2024-08-05 11:02:55--  https://raw.githubusercontent.com/tushortz/variety-bible-text/master/bibles/nasb.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4685837 (4.5M) [text/plain]
Saving to: ‘nasb.txt.1’

nasb.txt.1          100%[===================>]   4.47M  --.-KB/s    in 0.08s   

2024-08-05 11:02:55 (52.9 MB/s) - ‘nasb.txt.1’ saved [4685837/4685837]



In [ ]:
with open('nasb.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  4623633


Just under 5 million characters in nasb.txt file

Get a sorted list of unique string characters that are in the text

Vocabulary size is the number of these unique characters = 78 in total here

In [ ]:
# print(text[:1000])

chars = sorted(list(set(text)))
vocabulary = chars
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"'()*,-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz
78


In [ ]:
print(vocabulary)

['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Strategy to tokenise the input text

Tokenise = Convert raw text string to sequence of integers

Here, simply convert individual characters into integers as this is a character level model


encoder: (raw) input to tensor (each element in the tensor is an integer between 0 and vocabulary size - 1)

decoder: tensor to (reconstructed) input

Method:

Iterate over all the characters in the vocabulary

Create a lookup table

To encode a string

Translate all the characters in the string individually


In [ ]:
# simple tokenisation scheme index of sorted vocabulary (as this is character level)


# lookup tables: characters/tokens to integers/vectors and vice versa
characterToken_to_scalarTensor = { character : index for index, character in enumerate(vocabulary) }
scalarTensor_to_characterToken = { index : character for index, character in enumerate(vocabulary) }

# in case of issues down the line
stoi = characterToken_to_scalarTensor # string to integer
itos = scalarTensor_to_characterToken # integer to string

# takes in a string
# outputs a list of integers (list of scalar tensors)
# scalar tensor example = 23
encode = lambda string: [ characterToken_to_scalarTensor[character] for character in string ]

# takes in a list of scalar tensors
# outputs a string
decode = lambda vector: ''.join([ scalarTensor_to_characterToken[scalarTensor] for scalarTensor in vector ])


print(encode("hii there"))
print(decode(encode("hii there")))

[59, 60, 60, 1, 71, 59, 56, 69, 56]
hii there


Tokenise entire training set of the Bible

Using the PyTorch Library

Get all the text

Enocode it

Wrap it in a torch.tensor

Remember here each character is being tokenised

In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])

torch.Size([4623633]) <built-in method type of Tensor object at 0x7a560a5a9f80>
tensor([32, 65,  1, 71, 59, 56,  1, 53, 56, 58, 60, 65, 65, 60, 65, 58,  1, 30,
        66, 55,  1, 54, 69, 56, 52, 71, 56, 55,  1, 71, 59, 56,  1, 59, 56, 52,
        73, 56, 65, 70,  1, 52, 65, 55,  1, 71, 59, 56,  1, 56, 52, 69, 71, 59,
        10,  1,  9,  9,  1, 58, 56, 65, 56, 70, 60, 70,  1, 12, 21, 12,  0, 10,
         0, 43, 59, 56,  1, 56, 52, 69, 71, 59,  1, 74, 52, 70,  1, 57, 66, 69,
        64, 63, 56, 70, 70,  1, 52, 65, 55,  1, 73, 66, 60, 55,  8,  1, 52, 65,
        55,  1, 55, 52, 69, 62, 65, 56, 70, 70,  1, 74, 52, 70,  1, 66, 73, 56,
        69,  1, 71, 59, 56,  1, 70, 72, 69, 57, 52, 54, 56,  1, 66, 57,  1, 71,
        59, 56,  1, 55, 56, 56, 67,  8,  1, 52, 65, 55,  1, 71, 59, 56,  1, 42,
        67, 60, 69, 60, 71,  1, 66, 57,  1, 30, 66, 55,  1, 74, 52, 70,  1, 64,
        66, 73, 60, 65, 58,  1, 66, 73, 56, 69,  1, 71, 59, 56,  1, 70, 72, 69,
        57, 52, 54, 56,  1, 66, 57,  1, 

Seperate dataset into train, validation and testing split

Keep the validation data aside to ensure the model is not just "perfectly" memorising

Goal: Neural network that sort of creates Bible like text to hopefully understand the linguistic patterns within the Bible for deeper study

In [ ]:
train_index_cut = int( 0.64 * len(data) )
val_index_cut = int( 0.8 * len(data) )

# training data for the transformer
train_data = data[:train_index_cut]
val_data = data[train_index_cut:val_index_cut]
test_data = data[val_index_cut:]

Now plug some chunks of data into the Transformer

Sample random little chunks of the training set

Training a chunk at a time

Maximum chunk length is called block size in code (typically)

Sometimes block size is also called context length

In [ ]:
# start with a context_length aka block_size of 8

context_length = 8

block_size = context_length # to prevent any confusion later on

Context length number of examples of next token in this context (character) prediction, hence the length context_length+1

To be more precisie predicting position 2, 3, 4 all the way up to 9

In [ ]:
train_data[:context_length+1]

# 8 indi

tensor([32, 65,  1, 71, 59, 56,  1, 53, 56])

X = input to the transformer (of length context_length)

y = target for each position in the input X (still of length context_length)

Iterate over all parts of the context length

Think of it as the time dimension

In [ ]:
X = train_data[:context_length]
y = train_data[1:context_length+1]

print(f"Below are the {context_length} examples hidden in a chunk of {context_length+1} characters")
print("These have been sampled from the training set")
print(f"Here is the sample: {train_data[:context_length+1]}\n")

for position_index in range(context_length):
  # characters in X up to position_index and including position_index
  context = X[:position_index+1]
  target = y[position_index]
  print(f"when input is {context} the taget is: {target}")

Below are the 8 examples hidden in a chunk of 9 characters
These have been sampled from the training set
Here is the sample: tensor([32, 65,  1, 71, 59, 56,  1, 53, 56])

when input is tensor([32]) the taget is: 65
when input is tensor([32, 65]) the taget is: 1
when input is tensor([32, 65,  1]) the taget is: 71
when input is tensor([32, 65,  1, 71]) the taget is: 59
when input is tensor([32, 65,  1, 71, 59]) the taget is: 56
when input is tensor([32, 65,  1, 71, 59, 56]) the taget is: 1
when input is tensor([32, 65,  1, 71, 59, 56,  1]) the taget is: 53
when input is tensor([32, 65,  1, 71, 59, 56,  1, 53]) the taget is: 56


This gives the transformer familiarity with seeing different sized context sizes and still predicting the output correctly

Note: during inference, the transformer never receives more than the context length so trailing parts are truncated

The batch dimension

Stacked up chunks for efficiency

Keeps the GPUs busy, as they are good at parallel processing

TLDR; processing multiple chunks all at the same time (but also those chunks are processed independently, they do not talk to each other)

In [ ]:
# sampling random locations
# so to make experiments reproducible later on, set a random seed
torch.manual_seed(1337)

# the number of independent sequences we are processing
batch_size = 4

# maximum context length
block_size = 8

# how we get a batch for any arbitrary split
def get_batch(split):

  # generate a small batch of data of inputs x and targets y
  # this results in a data array
  data = train_data if split == "train" else val_data if split == "val" else test_data

  # generate random positions to grab a chunk out of
  # this is a random integer from 0 to length of data - block size (to prevent overshooting)
  # get batch size of number of these random integers (random offsets)
  # random offsets into the appropriate dataset (ie training set)
  ix = torch.randint(low=0, high=len(data)-block_size, size=(batch_size,)) # outputs a list/tensor

  x = torch.stack(
      [ data[ random_offset : random_offset + block_size] for random_offset in ix ]
  )

  # get the corresponding target
  y = torch.stack(
      [ data[ random_offset+1 : random_offset + block_size + 1 ] for random_offset in ix ]
  )

  # torch.stack: stack up as rows
  return x, y

# sample
# shape is rows=batch_size x column=block_sizeAkaContextLength
xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("---")

for batch_index in range(batch_size): # batch dimension
  for t in range(block_size): # time dimension (context length)
    # include the corresponding batch index
    context = xb[ batch_index, :t+1 ]
    target = yb[ batch_index, t ]
    print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[56, 54, 66, 65, 55,  1, 55, 52],
        [69,  1, 74, 60, 71, 59,  1, 60],
        [66, 70, 59, 72, 52,  1, 70, 52],
        [ 1, 54, 59, 52, 65, 58, 56, 55]])
targets:
torch.Size([4, 8])
tensor([[54, 66, 65, 55,  1, 55, 52, 76],
        [ 1, 74, 60, 71, 59,  1, 60, 71],
        [70, 59, 72, 52,  1, 70, 52, 60],
        [54, 59, 52, 65, 58, 56, 55,  1]])
---
when input is [56] the target: 54
when input is [56, 54] the target: 66
when input is [56, 54, 66] the target: 65
when input is [56, 54, 66, 65] the target: 55
when input is [56, 54, 66, 65, 55] the target: 1
when input is [56, 54, 66, 65, 55, 1] the target: 55
when input is [56, 54, 66, 65, 55, 1, 55] the target: 52
when input is [56, 54, 66, 65, 55, 1, 55, 52] the target: 76
when input is [69] the target: 1
when input is [69, 1] the target: 74
when input is [69, 1, 74] the target: 60
when input is [69, 1, 74, 60] the target: 71
when input is [69, 1, 74, 60, 71] the target: 59
when input is [69,